# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [6]:
import pandas as pd
from azureml.core import Workspace, Datastore, Dataset, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
# Read and clean the data

raw_data = pd.read_csv('data-unbalanced.csv')
raw_data = raw_data.dropna()
raw_data = pd.get_dummies(raw_data, columns=['PROD_CDE','PREP ASSOCIATE', 'FURN'])
print(raw_data.shape)

(26038, 93)


In [10]:
# Create a new Dataset from the raw_data
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')

dataset = Dataset.Tabular.register_pandas_dataframe(raw_data, datastore ,"Sinter", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/b22ba56f-425b-4554-95de-18018f960523/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [11]:
# Register the dataset
dataset = dataset.register(workspace = ws, name='Sinter_dataset', description='Sinter scrap dataset')

In [12]:
# choose a name for experiment
experiment_name = 'Sinter_AML'

experiment=Experiment(ws, experiment_name)

# Create a Copute target

In [13]:
cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [24]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HAS SCRAP AT SINTER",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [25]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


InvalidValueException: InvalidValueException:
	Message: Assertion Failed. Argument dataset_json is null. Target: dataset_json
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Encountered an internal AutoML error. Error Message/Code: InvalidValueException. Additional Info: InvalidValueException:\n\tMessage: Assertion Failed. Argument dataset_json is null. Target: dataset_json\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"message\": \"Assertion Failed. Argument dataset_json is null. Target: dataset_json\",\n        \"target\": \"dataset_json\"\n    }\n}",
        "details_uri": "https://aka.ms/automltroubleshoot",
        "target": "dataset_json",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "AutoMLInternal"
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
